In [1]:
'''
Obtiene letra de canciones
'''
import urllib.parse
import cancion_logica

# obtener canciones pendientes si letras registradas
canciones_pendientes = cancion_logica.obtenerCancionesPendientes()

for cancion_pendiente in canciones_pendientes:
   # obtiene la cancion y autor
   cancion = cancion_pendiente['cancion']
   cantante = cancion_pendiente['cantante'] 
   print('cancion: ' + cancion)
   print('cantante: ' + cantante)
   print(cancion_pendiente)
   
   # formatear canción y cantante como parámetro de URL
   cancion_code_url = urllib.parse.quote(cancion)
   cantante_code_url = urllib.parse.quote(cantante)
   print('cancion codificado para url: ' + cancion_code_url)
   print('cantante codificado para url: ' + cantante_code_url)

cancion: PELELE
cantante: MORAD
{'_id': ObjectId('630a2ce69ffbb04a4088c611'), 'cancion': 'PELELE', 'cantante': 'MORAD', 'estado': 'SIN LETRA', 'letra': 'letra prueba 6'}
cancion codificado para url: PELELE
cantante codificado para url: MORAD


In [2]:
# prepara URL de Lyrics para buscar letras por cancion y autor


WEB_LYRICS = "https://www.lyrics.com/lyrics/"
cancion_autor_code_url = urllib.parse.quote(cancion_code_url + ' ' + cantante_code_url)
url_busqueda_lyrics = '%s%s' % (WEB_LYRICS, cancion_autor_code_url) 
print(url_busqueda_lyrics)

https://www.lyrics.com/lyrics/PELELE%20MORAD


In [3]:
import requests
from bs4 import BeautifulSoup

# Obtiene del contenido web la letra de la canción
page = requests.get(url_busqueda_lyrics)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="content-body")
job_elements = results.find_all("div", class_="sec-lyric clearfix")
# obtiene elementos del autor, cancion y enlace donde está la letra de la canción
url_letra_cancion = ''
for job_element in job_elements:
    cancion_elem = job_element.find("p", class_="lyric-meta-title")
    autor_elem = job_element.find("p", class_="lyric-meta-album-artist")
    
    if autor_elem == None:
        autor_lyrics = ''
    else:
        autor_lyrics = autor_elem.text

    cancion_lyrics = cancion_elem.text
    ref = job_element.find("pre", class_="lyric-body")

    print("cancion lyrics: ", cancion_lyrics)
    print("autor lyrics: ", autor_lyrics)
    print("url letra lyrics: ", ref.attrs['onclick'])
    print("cancion bdd: " + cancion)
    print("autor bdd: " + cantante)
    print('-----------------')

    # valida autor y canción para obtener el enlace a la letra de la canción
    if cancion_lyrics.casefold() == cancion.casefold() and autor_lyrics.casefold() == cantante.casefold():
        url_letra_cancion =  ref.attrs['onclick']
        print (url_letra_cancion)
       

if url_letra_cancion == '':
    print ('NO ENCONTRO LETRA')
else:
    url_letra_cancion = url_letra_cancion.split('=')[1].replace('\'','')
    print(url_letra_cancion)


cancion lyrics:  Y.N.M.S.
autor lyrics:  
url letra lyrics:  location.href='https://www.lyrics.com/lyric-lf/7637049/7aared/Y.N.M.S.';
cancion bdd: PELELE
autor bdd: MORAD
-----------------
cancion lyrics:  Pelele
autor lyrics:  Morad
url letra lyrics:  location.href='https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele';
cancion bdd: PELELE
autor bdd: MORAD
-----------------
location.href='https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele';
cancion lyrics:  2x1
autor lyrics:  Morad
url letra lyrics:  location.href='https://www.lyrics.com/lyric-lf/2251955/Morad/2x1';
cancion bdd: PELELE
autor bdd: MORAD
-----------------
https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele;


In [5]:
# obtiene contenido de la página web de la letra de la canción
page_letra_cancion = requests.get(url_letra_cancion)
soup_letra_cancion = BeautifulSoup(page_letra_cancion.content, "html.parser")
# obtiene el cuerpo del contenido web de la letra de la canción
results_letra_cancion = soup_letra_cancion.find(id="content-body")
job_elements_letra = results_letra_cancion.find_all("div", class_="lyric clearfix")
for job_element in job_elements_letra:
    letra = job_element.find("pre", id="lyric-body-text")
print(letra.text)

Hombre, no te sale (sale) (Tr. Fact)
Hombre, no te sale (sale, sale, sale)
Morad, no te, Morad cómo te sales (sale', sale, sale, sale)
Si entras aquí no, ah (sale', sale, sale', sale', sale')

Morad cómo te sales (sales), si entras aquí no sales (sales)
Sabes que tú no vales (vale'), como sueno yo en instrumentales (-ale')
Hablando de los mundo' de ilegale' (-ale')
Pegadizo como Dale Don Dale (-ale)
Yo sonando siempre en la calle, tú 40 Principales (-ale')

Cuatro guitarrista' (-ista'), 17 que tienen orquesta (orquesta)
Para hacer un tema (ala) que no suena ni en la fiesta (no, no, no)
Yo hago mi escritura fumando uno de verdura (¿ah?)
Tú dos meses duras, tú está' fuera 'e cobertura (jaja)

La calle está loca (loca), eso e' lo que pica (pica)
Pagamos a Toca y tú pidiéndole a tu chica (chica)
El capo González (-ález) puede que te cale (cale)
Tu coche se cala, y acaba' luego en la tele (cuidao')

Mira que ya e' fácil (cuidao'), en cinco minuto' (cuidao')
Lo hago ya duro, rápido y también

In [6]:
# actualiza la letra en la colección de la canción
resultado_final = cancion_logica.actualizarCancionPendiente(cancion_pendiente, letra.text)
print(resultado_final)

documento actualizado correctamente
